# 2.2.9 Estimación de estadísticos usando bootstrap

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.utils import resample

from scipy import stats

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

La estimación de la distribución de probabilidades (y sus propiedades) de un estadístico suele ser dificil debido a que solo unos pocos estadísticos siguen una distribución de probabilidades conocida. El bootstrap es un método numérico para realizar está estimación a partir de una muestra de datos. Uno de sus usos más importantes es la estimación del error promedio de pronóstico en modelos predictivos comúnmente usados en analítica.

## Descripción del problema

Se tiene la siguiente muestra de datos, obtenida de un experimento. Se desea estimar la media de la población.

In [ ]:
x = [0.09, 0.15, 0.18, 0.16, 0.23, 0.35, 0.49, 0.68, 0.71, 0.85, 0.96, 0.98, 0.27]

## Conceptos básicos

A continuación se presentan las principales definiciones asociadas al concepto de bootstrap.

![Alt text](images/bootstrap_1.png)

**Población:** Colección de objetos o individuos sobre los cuales se desea obtener información de interés.

**Muestra:** Grupo obtenido aleatoriamente que es representativo de la población.

**Parámetro:** Un valor que resume una propiedad o característica de la población entera.

**Muestreo sin reemplazo:** Cuando una observación es seleccionada ya no está disponible en adelante.

**Muestreo con reemplazo:** Cada observación es devuelta a la población después de ser seleccionada.

**Muestreo aleatorio:** Muestreo de elementos de forma aleatoria tal que cada miembro de la población tiene igual probabilidad de ser seleccionado.

**Muestreo estratificado:** División de la población en estratos y muestreo aleatorio de cada estrato.

**Muestra aleatoria simple:** Muestra que resulta de muestrear la población sin estratificar.

**Muestra sesgada:** Muestra que no representa la población.

**Sesgo estadístico:** Errores de muestreo o medición que son sistemáticos y producidos por la medición o el muestreo mismo.

**Estadístico:** Un número que resume una propiedad o característica de la muestra.

**Estadístico muestral:** Una métrica calculada para una muestra de datos obtenida de forma aleatoria de la población.

**Distribución muestral:** Es la distribución de probabilidad de un estadístico muestral calculado sobre muchas muestras o remuestreos.

**Error estándar:** Es la desviación estándar del estadístico muestral sobre muchas muestras.

Para estimar el valor de un parámetro de la población mediante un estadístico calculado sobre la muestra, se deben tener en cuenta alguna de las dos siguientes opciones:

- Pruebas de hipótesis (se caracterizaron en el módulo de análisis exploratorio).
- Estimación de intervalos de confianza.

El problema abordado aquí radica en que el estadístico calculado (la media de la muestra) es una aproximación al parámetro real de la población (la media de la población); si se obtienen diferentes muestras de datos, y se calcula el estadístico de interés, se obtendrán distintos valores (distintas medias); entonces, la pregunta es cómo aproximar la media de la población a partir de la muestra de datos. Esta es la utilidad del bootstrapping.

## Bootstrapping

El bootstrap se usa cuando no es posible derivar ecuaciones que permitan aproximar la distribución de probabilidades de un estadístico. El bootstrap se basa en el remuestreo de los datos para poder obtener una muestra del estadístico que se está calculado.

Suponga que tiene una muestra de ocho ejemplos:

$$

\{x1,x2,x3,x4,x5,x6,x7,x8\}

$$

Una muestra bootstap se obtiene de la muestra original, seleccionando ocho elementos de forma aleatoria con reemplazo. Por ejemplo, una muestra bootstrap podría ser:

$$

\{x1,x2,x2,x2,x4,x1,x7,x7\}

$$

Nóte que los elementos pueden repetirse. Sobre cada muestra bootstrap obtenida, se realiza el proceso de cómputo y se obtiene el estadístico de interés. Si este procedimiento se repite, por ejemplo, 500 veces para calcular el estadístico de interés, se tendrían 500 valores posibles de éste. Así, una estimación mucho mejor del valor del estadístico, sería calcular su valor promedio a partir de la muestra de 500 valores. Más aún, podría calcularse el histograma o la distribución de probabilidades de dicho estadístico, lo cual es mucho más informativo.

## Algoritmo bootstrap no paramétrico

- Se tiene la muestra $x=\{xi;1,...,n\}$, que en ese caso es $\{x1,x2,x3,x4,x5,x6,x7,x8\}$ con n=8.
- Se seleccionan B muestras bootstrap independientes $x∗^1,...,x∗^B$ cada una de n elementos obtenidos de la muestra original. Por ejemplo:

$$x∗^1=\{x_1,x_2,x_2,x_2,x_4,x_1,x_7,x_7\}$$

$$x∗^2=\{x_4,x_1,x_2,x_4,x_8,x_3,x_1,x_1\}$$

$$x∗^3=\{x_7,x_7,x_1,x_3,x_6,x_1,x_4,x_8\}$$

y así sucesivamente.

- Para cada muestra bootstap se calcula el estadístico de interes: $θ(b)=t(x∗b)$. Es decir:

$$θ(1)=t(x∗^1)=t(\{x_1,x_2,x_2,x_2,x_4,x_1,x_7,x_7\})$$

$$θ(2)=t(x∗^2)=t(\{x_4,x_1,x_2,x_4,x_8,x_3,x_1,x_1\})$$

$$θ(3)=t(x∗^3)=t(\{x_7,x_7,x_1,x_3,x_6,x_1,x_4,x_8\})$$

y así sucesivamente.

- La muestra $θ(1),...,θ(B)$ representa la distribución de probabilidades de θ y puede calcularsele valor esperado, desviación estándar, etc. El valor buscado sería:


$$ \bar{θ}(∙) = \frac{1}{B}\sum_{b=1}^{B}θ(b) $$

El error estándar se puede calcular como:

$$ se_B=\left \{\frac{1}{B−1} \sum_{b=1}^{B} [θ(b)−\bar{θ}(∙)]^2 \right \}^{1/2} $$

## Cómputo en Python

A continuación se presenta la solución del problema usando Python.

In [ ]:
#
# La función resample permite hacer un muestreo con reemplazo.
#
boot = resample(
    x,  # muestra
    replace=True,  # reemplazo?
    n_samples=len(x),  # longitud de la nueva muestra
    random_state=None,
)  # generador de aleatorios

boot

In [ ]:
##
## Se generan 500 muestras bootstrap
##
sample = [resample(x, replace=True, n_samples=len(x)) for _ in range(500)]
sample[0:2]

In [ ]:
#
# Se computa la media para cada muestra bootstrap
#
stat = [np.mean(s) for s in sample]
stat[0:5]

In [ ]:
#
# Estimado de la media de la población
#
np.mean(stat)

In [ ]:
#
# Histograma de las medias calculadas
#
sns.set(color_codes=True)
sns.distplot(stat, rug=True)
plt.show()

In [ ]:
#
# Error estándar (desviación estándar) para
# el valor computado de la media
#
np.std(stat)

## Distribución de probabilidades del estadístico boostrap

Se obtiene aplicando el teorema del Límite Central.

Si $s_1 ,…, s_n$ es una muestra aleatoria donde las observaciones si son independientes e identicamente distribuidas de una distribución con media μ y varianza finita $\sigma^2$, entonces:

$$ \bar{S}(∙) = \frac{1}{n}\sum_{i=1}^{n}s_i $$

sigue una distribución normal con media $\mu$ y varianza $(\sigma_s)^2=\frac{\sigma^2}{n}$.

Note que no se hace ningún supuesto sobre la distribución que siguen las observaciones $s_i$

En este caso, $s_i=θ(i)$, lo que quiere decir que $\theta(i)$ sigue una distribución normal $N(\mu, \sigma^2)$ con:

$$ \mu=\bar{θ}(∙) =\frac{1}{B} \sum_{b=1}^{B} θ(b) $$

y

$$ \sigma = se_B=\left \{\frac{1}{B−1} \sum_{b=1}^{B} [θ(b)−\bar{θ}(∙)]^2 \right \}^{1/2}$$

In [ ]:
#
# Distribución normal teórica
#
sns.set(color_codes=True)
sns.distplot(stat, rug=True, fit=stats.norm)
plt.show()

In [ ]:
#
# Intervalos de confianza para +/- 2 sigma
# aprox el 95% de confianza
#
[np.mean(stat) - 2 * np.std(stat), np.mean(stat) + 2 * np.std(stat)]

## Jacknife

Se obtiene al dejar un solo elemento fuera de la muestra de datos.

- Se tiene la muestra $x=\{x_i;1,...,n\}$, que en ese caso es $\{x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8\}$ con n=8.
- Se obtienen n muestras jackknife $x(i), i=1,...,n$, donde:

$$ x_{(i)}=(x_1,x_2,...,x_{i−1},x_{i+1},...,x_n)$$

Es decir:

$$x_{(1)}=(x_2,x_3,x_4,...,x_7,x_8)$$

$$x_{(2)}=(x_1,x_3,x_4,...,x_7,x_8)$$

$$x_{(3)}=(x_1,x_2,x_4,...,x_7,x_8)$$

$$...$$

$$x_{(8)}=(x_1,x_2,x_3,...,x_6,x_7)$$

- Se calcula el estadístico para la muestra jackknife:

$$ θ_{(i)}=t(x_{(i)}) $$

que para la muestra actual es:

$$θ_{(1)}=t(x_{(1)})$$

$$θ_{(2)}=t(x_{(2)})$$

$$...$$

$$θ_{(8)}=t(x_{(8)})$$

El error estándar se obtiene como:

$$ se_{jack} = \left \{\frac{n+1}{n} \sum_{i=1}^{n} [θ(i)−{θ}(∙)]^2 \right \}^{1/2}$$

$$ \bar{θ}(∙) =\frac{1}{n} \sum_{i=1}^{n} θ_{(i)} $$

In [ ]:
print('ok_')